# DATASCI W261: Machine Learning at Scale
## Assignment Week 1
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Due date 19-Jan-2016

### HW1.0.0: Define big data. Provide an example of a big data problem in your domain of expertise. 

"Big data" is a broad term that has many interpretations. It doesn't seem fair to assign a concrete size barrier above which is considered "big," and below which is not. A definition that resonates with me is that if a data set is too large to fit on or process with one computer in a reasonable amount of time, then it is considered big data. This definition allows for more flexibility as hard drives become larger and computers become more powerful.

I currently work at a home security company, so we are constantly generating IoT-type data. Analysis on any of the data we generate from wireless sensors is usually a big data problem, unless we are looking at a tiny subset of data. One of the most important problems we are currently trying to solve is determining when a home is unoccupied using data from wireless sensors such as motion detectors and door sensors. It is relatively easy to determine when a home is occupied: we can be fairly certain that someone is home when motion is detected. Conversely, the absence of motion does not always indicate that the home is unoccupied. A simple example is at nighttime when people are usually sleeping.

### HW1.0.1: In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreducible error for a test dataset T when using polynomial regression models of degree 1, 2, 3, 4, 5 are considered. How would you select a model?

In order to estimate the bias, variance and irreducible error (noise) for a single data that is generated from the unknown true function $f(x)$, we first need to generate multiple sets of data from our original data set T. We can do this by sampling the original data set with replacement (bootstrapping). If we repeat the bootstrap resample 50 times, we will have 50 data sets to work with.

With each of the 50 new data sets, we split the data set into training set and a testing set. We fit polynomials of degree 1, 2, 3, 4 and 5 to the training set. This yields 50 models per polynomial degree. For each polynomial degree, we can then estimate the average variance and bias using the testing set.

#### Variance estimation
For each observation, $x$, in the testing set, we now have 50 predictions per polynomial degree, which we denote as $y_1, y_2, ..., y_{50}$. We find the average of these predictions, denoted as $\bar{y}$. We can find the variance of the predictions using the formula: $E((y_i-\bar{y})^2)$. We then average the variance for each data point in the testing set, and then repeat the process for each polynomial degree. Thus, we will have one average variance per polynomial degree.

#### Bias estimation
For each observation, $x$, we also have the actual value of $y$ in the testing set. The bias of the observation $x$ is the difference between the average prediction and the actual value: $\bar{y}-y$. If we happened to have the true function $f(x)$ from which the data were generated, we would calculate the bias as: $\bar{y}-f(x)$.

We then average the bias for each data point in the testing set per polynomial degree, and repeat the process for each polynomial degree. This yields one average bias per polynomial degree. It is also useful to calculate the average squared bias, which is calculated by squaring the bias before averaging all of the observations in the testing set.

#### Noise estimation
If we do not know the true function $f(x)$, we are forced to make the assumption that the noise is zero. If we knew the true function $f(x)$ that the data set T was generated from, we would be able to calculate the irreducible error, which would be the square of the difference between the observed values and the true function: $(y-f(x))^2$.

#### Model selection
We know that there is a trade-off between bias and variance. As the model gets more complex, bias generally decreases, while variance generally increases. We can plot the sum of the squared bias and the variance, and choose the degree where the sum is minimized.
